# Tutorial: Using `graphs.py`

This example notebook demonstrates the main plotting and analysis utilities provided in `graphs.py`. Each section generates a small synthetic dataset, calls the target function, and highlights the expected outputs. Replace the demo data with your own observations for real‐world analyses.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy
import sys
sys.path.append("../../src/")
# Local import of graphs.py (make sure it is on your PYTHONPATH or in the same folder)
import micromet
%matplotlib inline

## Generate synthetic datasets

In [ ]:
# --- Energy balance demo data -----------------------------------------
dates = pd.date_range('2024-06-18 00:00', '2024-06-20 00:00', freq='30T', tz='UTC')
hours = dates.hour + dates.minute / 60
SW_IN = np.maximum(0, 900 * np.sin(np.deg2rad((hours - 6) / 24 * 360)))
LW_IN = np.full_like(SW_IN, 350)
SW_OUT = SW_IN * 0.05
LW_OUT = LW_IN * 0.9
NETRAD = SW_IN + LW_IN - SW_OUT - LW_OUT
G = 0.10 * NETRAD
LE = 0.40 * (NETRAD - G)
H = 0.50 * (NETRAD - G)

energy_df = pd.DataFrame(
    {
        'SW_IN': SW_IN,
        'LW_IN': LW_IN,
        'SW_OUT': SW_OUT,
        'LW_OUT': LW_OUT,
        'NETRAD': NETRAD,
        'G': G,
        'LE': LE,
        'H': H,
    },
    index=dates,
)
energy_df.head()

## 1. Energy Sankey Diagram

In [ ]:
fig = micromet.energy_sankey(energy_df, '2024-06-19 12:00')
fig.show()

## 2. Instrument scatter comparison

In [ ]:
# --- Synthetic instrument data ---------------------------------------
rng = pd.date_range('2024-06-01', periods=1000, freq='10min', tz='UTC')
edmet = pd.DataFrame(index=rng)
edmet['Temp_Instrument1'] = 20 + np.random.normal(0, 1, size=len(rng))
edmet['Temp_Instrument2'] = edmet['Temp_Instrument1'] + np.random.normal(0, 0.5, size=len(rng))

compare_dict = {
    'Temp_Instrument1': ('Temperature', 'Instrument 1', '°C'),
    'Temp_Instrument2': ('Temperature', 'Instrument 2', '°C'),
}

station = 'DemoStation'
results = micromet.scatterplot_instrument_comparison(edmet, compare_dict, station)
results[:3]  # slope, intercept, R²

## 3. Bland–Altman Plot

In [ ]:
_ = micromet.bland_alt_plot(edmet, compare_dict, station)

## 4. Mean Difference Plot & Error Metrics

In [ ]:
series1 = edmet['Temp_Instrument1'][::6]
series2 = edmet['Temp_Instrument2'][::6]

# Mean squared error
mse = micromet.mean_squared_error(series1, series2)
print(f'MSE = {mse:.3f}')

# Mean difference (Tukey) plot
_ = micromet.mean_diff_plot(series1, series2)

## 5. Time‑series plot with date range

In [ ]:
# --- Build a MultiIndex dataset --------------------------------------
multi_index = pd.MultiIndex.from_product([
    ['DemoStation'], rng
], names=('station', 'datetime'))

multidf = pd.DataFrame({'Temperature': edmet['Temp_Instrument1'].values}, index=multi_index)

micromet.plot_timeseries_daterange(multidf, 'DemoStation', 'Temperature', '2024-06-05', '2024-06-07')

## 6. Saving plots interactively

Use `ipywidgets` to attach the `graphs.save_plot` callback to a button:
```python
from ipywidgets import Button
btn = Button(description='Save Current Figure')
btn.on_click(graphs.save_plot)
btn
```

## Conclusion
This notebook showcased each of the core functions in **graphs.py**. Adapt the synthetic data creation steps to load your real datasets, and integrate these plotting utilities into your own analysis workflows.